## Evironment Setup

In [ ]:
!pip install -q transformers datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00


In [ ]:
#w/o GPU for CUDA check
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!nvidia-smi

Mon Jan 29 05:28:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# CUDA_LAUNCH_BLOCKING = "1"


In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import pickle
import transformers
import json
import copy
from os import listdir
from PIL import Image
import pprint

from tqdm import tqdm
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader

from transformers import LxmertTokenizer, LxmertModel
tokenizer = LxmertTokenizer.from_pretrained('unc-nlp/lxmert-base-uncased')

import torch.nn.functional as F
from transformers import AdamW, AutoModel, set_seed
import logging
logging.basicConfig(level=logging.ERROR)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/00_torino_adnlp/

/content/drive/MyDrive/00_torino_adnlp


| FormNLU     | Images | Title | Section | Table-Key | Table-Value | Form-Key | Form-Value |
|-------------|--------|-------|---------|-----------|-------------|----------|------------|
| Train       | 464+71 (535) |  1068 |    1560 |      2669 |        2672 |     3708 |       3568 |
| Val         | 66+10 (76)  |   152 |     215 |       380 |         379 |      524 |        510 |
| Test        | 131+15 (146) |   292 |     418 |       730 |         730 |     1009 |        978 |
| Printed     |     50 |    98 |     148 |       250 |         249 |      346 |        332 |
| HandWritten |     50 |   100 |     150 |       249 |         226 |      348 |        315 |

In [ ]:
# open json files

with open('formnlu_features/formnlu_all_feat_train.pkl', 'rb') as f:
    train_all_feat = pickle.load(f)

with open('formnlu_features/formnlu_all_feat_val.pkl', 'rb') as f:
    val_all_feat = pickle.load(f)

with open('formnlu_features/formnlu_all_feat_test.pkl', 'rb') as f:
    test_all_feat = pickle.load(f)

In [ ]:
print(train_all_feat.keys())
print(train_all_feat['01502814_page-0'].keys())
print(train_all_feat['01502814_page-0']['objects'][0].keys())
print(len(train_all_feat.keys()))

print(val_all_feat.keys())
print(val_all_feat['00910692_page-1'].keys())
print(val_all_feat['00910692_page-1']['objects'][0].keys())
print(len(val_all_feat.keys()))

print(test_all_feat.keys())
print(test_all_feat['00917958_page-0'].keys())
print(test_all_feat['00917958_page-0']['objects'][0].keys())
print(len(test_all_feat.keys()))

dict_keys(['01502814_page-0', '01311266_page-0', '01312096_page-0', '01314156_page-0', '01322021_page-0', '01327411_page-0', '01327879_page-0', '01330181_page-0', '01330743_page-0', '01331561_page-0', '01333748_page-0', '01334844_page-0', '01339499_page-0', '01342494_page-0', '01344731_page-0', '01346455_page-0', '01355023_page-0', '01359605_page-1', '01359608_page-1', '01368409_page-0', '01369341_page-0', '01370460_page-0', '01370855_page-0', '01372560_page-0', '01373528_page-0', '01376050_page-0', '01381847_page-0', '01382283_page-0', '01382666_page-0', '01383077_page-0', '01384450_page-0', '01385004_page-1', '01393056_page-0', '01395037_page-0', '01396709_page-0', '01397263_page-0', '01398740_page-0', '01398791_page-1', '01400324_page-0', '01403379_page-0', '01406011_page-0', '01407789_page-0', '01408127_page-1', '01408579_page-0', '01408937_page-1', '01413192_page-0', '01421331_page-0', '01421595_page-0', '01424210_page-0', '01424827_page-0', '01425017_page-0', '01432424_page-0', '

In [ ]:
# original mapping

# category_mapping = {1: 'Title', 2: 'Section', 3: 'Other', 4: 'Form_key', 5: 'Form_value', 6: 'Table_key', 7: 'Table_value'}
# label_dict = {v: k for k, v in category_mapping.items()} #label2id
# label_list = list(label_dict.keys())

# print("label_dict", label_dict)
# print("label_list", label_list)

# id2label = {k: v for k,v in enumerate(label_list)}

label_dict {'Title': 1, 'Section': 2, 'Other': 3, 'Form_key': 4, 'Form_value': 5, 'Table_key': 6, 'Table_value': 7}
label_list ['Title', 'Section', 'Other', 'Form_key', 'Form_value', 'Table_key', 'Table_value']


In [ ]:
# new mapping starting from 0
# label_dict = {}
# label_id = 0

# for doc in val_all_feat:
#   objects = val_all_feat[doc]['objects']
#   for i in objects:
#     #print(objects[i]['category_name'])
#      if objects[i]['category_name'] not in label_dict:
#       label_dict[objects[i]['category_name']] = label_id
#       label_id += 1

# print(label_dict)

{'Title': 0, 'Other': 1, 'Section': 2, 'Form_key': 3, 'Table_key': 4, 'Table_value': 5, 'Form_value': 6}


#### debugging code (train - empty data exist)

##### example code

In [ ]:
train = {
    '011': {
        'name': 'example_image.jpg',
        'objects': {
            0: {'global_id': 142, 'image_id': 1, 'bbox': [10, 20, 30, 40], 'text': 'Object1', 'category_id': 2},
            1: {'global_id': 143, 'image_id': 1, 'bbox': [20, 30, 40, 50], 'text': 'Object2', },
            2: {'global_id': 144, 'image_id': 1, 'bbox': [20, 30, 40, 50], 'text': 'Object3', 'category_id': 3},
        }
    },
    '012': {
        'name': 'another_image.jpg',
        'objects': {
            0: {'global_id': 145, 'image_id': 2, 'bbox': [30, 40, 50, 60], 'text': 'Object4', 'category_id': 1},
            1: {'global_id': 146, 'image_id': 2, 'bbox': [40, 50, 60, 70], 'text': 'Object5', 'category_id': 2},
            2: {'global_id': 147, 'image_id': 2, 'bbox': [50, 60, 70, 80], 'text': 'Object6', },
        }
    }
}

# Iterate through each key in the 'train' dictionary
for key, data in train.items():
    # Filter out objects with missing 'category_id' or any other key you want to check
    data['objects'] = {idx: obj for idx, obj in data.get('objects', {}).items() if 'category_id' in obj and 'text' in obj}

# Print the updated 'train' dictionary
print(train)

{'011': {'name': 'example_image.jpg',
  'objects': {0: {'global_id': 142,
    'image_id': 1,
    'bbox': [10, 20, 30, 40],
    'text': 'Object1',
    'category_id': 2},
   2: {'global_id': 144,
    'image_id': 1,
    'bbox': [20, 30, 40, 50],
    'text': 'Object3',
    'category_id': 3}}},
 '012': {'name': 'another_image.jpg',
  'objects': {0: {'global_id': 145,
    'image_id': 2,
    'bbox': [30, 40, 50, 60],
    'text': 'Object4',
    'category_id': 1},
   1: {'global_id': 146,
    'image_id': 2,
    'bbox': [40, 50, 60, 70],
    'text': 'Object5',
    'category_id': 2}}}}

In [ ]:
train = {
    '011': {
        'name': 'example_image.jpg',
        'objects': {
            0: {'global_id': 142, 'image_id': 1, 'bbox': [10, 20, 30, 40], 'text': 'Object1', 'category_id': 2},
            1: {'global_id': 143, 'image_id': 1, 'bbox': [20, 30, 40, 50], 'text': 'Object2', 'category_id': None},
            2: {'global_id': 144, 'image_id': 1, 'bbox': [20, 30, 40, 50], 'text': 'Object3', 'category_id': 3},
        }
    },
    '012': {
        'name': 'another_image.jpg',
        'objects': {
            0: {'global_id': 145, 'image_id': 2, 'bbox': [30, 40, 50, 60], 'text': 'Object4', 'category_id': 1},
            1: {'global_id': 146, 'image_id': 2, 'bbox': [40, 50, 60, 70], 'text': 'Object5', 'category_id': 2},
            2: {'global_id': 147, 'image_id': 2, 'bbox': [50, 60, 70, 80], 'text': 'Object6', 'category_id': None},
        }
    }
}

# Iterate through each key in the 'train' dictionary
for key, data in train.items():
    # Filter out objects with missing values in 'category_id' or any other key you want to check
    data['objects'] = {idx: obj for idx, obj in data.get('objects', {}).items() if all(obj.get(k) is not None for k in ['category_id', 'text'])}

# Print the updated 'train' dictionary
print(train)

{'011': {'name': 'example_image.jpg',
  'objects': {0: {'global_id': 142,
    'image_id': 1,
    'bbox': [10, 20, 30, 40],
    'text': 'Object1',
    'category_id': 2},
   2: {'global_id': 144,
    'image_id': 1,
    'bbox': [20, 30, 40, 50],
    'text': 'Object3',
    'category_id': 3}}},
 '012': {'name': 'another_image.jpg',
  'objects': {0: {'global_id': 145,
    'image_id': 2,
    'bbox': [30, 40, 50, 60],
    'text': 'Object4',
    'category_id': 1},
   1: {'global_id': 146,
    'image_id': 2,
    'bbox': [40, 50, 60, 70],
    'text': 'Object5',
    'category_id': 2}}}}

##### debug train set

if use orginal train data : caused an error ude to empty lines (e.g. KeyError: 'bert_cls')

In [ ]:
#objects = train_all_feat['00827863_page-1']['objects']
#for obj in objects:
#  print(objects[obj]['category_id'])

In [ ]:
train_sample = copy.deepcopy(train_all_feat)
print(train_sample.keys())
print(train_sample['01502814_page-0'].keys())
print(train_sample['01502814_page-0']['objects'][0].keys())
print(len(train_sample.keys()))

dict_keys(['01502814_page-0', '01311266_page-0', '01312096_page-0', '01314156_page-0', '01322021_page-0', '01327411_page-0', '01327879_page-0', '01330181_page-0', '01330743_page-0', '01331561_page-0', '01333748_page-0', '01334844_page-0', '01339499_page-0', '01342494_page-0', '01344731_page-0', '01346455_page-0', '01355023_page-0', '01359605_page-1', '01359608_page-1', '01368409_page-0', '01369341_page-0', '01370460_page-0', '01370855_page-0', '01372560_page-0', '01373528_page-0', '01376050_page-0', '01381847_page-0', '01382283_page-0', '01382666_page-0', '01383077_page-0', '01384450_page-0', '01385004_page-1', '01393056_page-0', '01395037_page-0', '01396709_page-0', '01397263_page-0', '01398740_page-0', '01398791_page-1', '01400324_page-0', '01403379_page-0', '01406011_page-0', '01407789_page-0', '01408127_page-1', '01408579_page-0', '01408937_page-1', '01413192_page-0', '01421331_page-0', '01421595_page-0', '01424210_page-0', '01424827_page-0', '01425017_page-0', '01432424_page-0', '

In [ ]:
# Iterate through each key in the 'train' dictionary
for key, data in train_sample.items():
    # Filter out objects with missing values in 'category_id' or any other key you want to check
    data['objects'] = {idx: obj for idx, obj in data.get('objects', {}).items() if all(obj.get(k) is not None for k in ['category_name', 'bert_cls', 'text'])}

In [ ]:
# sample
print(train_sample.keys())
print(train_sample['00842751_page-0'].keys())
print(train_sample['00842751_page-0']['objects'][0].keys())
print(len(train_sample['00842751_page-0']['visual_list']))
print(len(train_sample['00842751_page-0']['objects']))


dict_keys(['01502814_page-0', '01311266_page-0', '01312096_page-0', '01314156_page-0', '01322021_page-0', '01327411_page-0', '01327879_page-0', '01330181_page-0', '01330743_page-0', '01331561_page-0', '01333748_page-0', '01334844_page-0', '01339499_page-0', '01342494_page-0', '01344731_page-0', '01346455_page-0', '01355023_page-0', '01359605_page-1', '01359608_page-1', '01368409_page-0', '01369341_page-0', '01370460_page-0', '01370855_page-0', '01372560_page-0', '01373528_page-0', '01376050_page-0', '01381847_page-0', '01382283_page-0', '01382666_page-0', '01383077_page-0', '01384450_page-0', '01385004_page-1', '01393056_page-0', '01395037_page-0', '01396709_page-0', '01397263_page-0', '01398740_page-0', '01398791_page-1', '01400324_page-0', '01403379_page-0', '01406011_page-0', '01407789_page-0', '01408127_page-1', '01408579_page-0', '01408937_page-1', '01413192_page-0', '01421331_page-0', '01421595_page-0', '01424210_page-0', '01424827_page-0', '01425017_page-0', '01432424_page-0', '

In [ ]:
# original
print(train_all_feat['00842751_page-0'].keys())
print(train_all_feat['00842751_page-0']['objects'][0].keys())
print(len(train_all_feat['00842751_page-0']['visual_list']))
print(len(train_all_feat['00842751_page-0']['objects']))

dict_keys(['name', 'height', 'width', 'objects', 'object_list', 'visual_list'])
dict_keys(['global_id', 'image_id', 'bbox', 'text', 'category_id', 'category_name', 'bert_cls'])
31
31


In [ ]:
# original
for i in train_all_feat['00842751_page-0']['objects']:
  print(train_all_feat['00842751_page-0']['objects'][i]['text'])

   Form 604
     Corporations Act 2001
Section  671B
Notice of change of interests of substantial holder
Previous notice
Present notice
1.  Details  of  substantial  holder(1)
2.    Previous  and  present  voting  power
3.    Changes  in  relevant  interests
The total number of votes attached to all the voting shares in the company or voting interests in the scheme that the substantial holder or an associate (2) had a relevant interest (3)
in when last required, and when now required, to give a substantial holding notice to the company or scheme, are as follows:
To   Company Name/Scheme
ACN/ARSN
Name
ACN/ARSN (if applicable)
There was a change in the interests of the
substantial holder on
The previous notice was given to the company on
The previous notice was dated
Class of securities (4)
Person's votes
Voting power (5)
Person's votes
Voting power (5)





/        /
/        /
/         /





In [ ]:
#after drop the empty line
for i in train_sample['00842751_page-0']['objects']:
  print(train_sample['00842751_page-0']['objects'][i]['text'])

   Form 604
     Corporations Act 2001
Section  671B
Notice of change of interests of substantial holder
Previous notice
Present notice
1.  Details  of  substantial  holder(1)
2.    Previous  and  present  voting  power
3.    Changes  in  relevant  interests
The total number of votes attached to all the voting shares in the company or voting interests in the scheme that the substantial holder or an associate (2) had a relevant interest (3)
in when last required, and when now required, to give a substantial holding notice to the company or scheme, are as follows:
To   Company Name/Scheme
ACN/ARSN
Name
ACN/ARSN (if applicable)
There was a change in the interests of the
substantial holder on
The previous notice was given to the company on
The previous notice was dated
Class of securities (4)
Person's votes
Voting power (5)
Person's votes
Voting power (5)


In [ ]:
idx = 0

key = list(train_sample.keys())[idx]
page_info = train_sample[key]
print("key", key)
objects = page_info["objects"]
print(len(objects))


key 01502814_page-0
32


In [ ]:
# original & new mapping
# category_mapping = {1: 'Title', 2: 'Section', 3: 'Other', 4: 'Form_key', 5: 'Form_value', 6: 'Table_key', 7: 'Table_value'}
# {'Title': 0, 'Other': 1, 'Section': 2, 'Form_key': 3, 'Table_key': 4, 'Table_value': 5, 'Form_value': 6}

label_dict = {}
label_id = 0

for doc in train_sample:
  objects = train_sample[doc]['objects']
  for i in objects:
    #print(objects[i]['category_name'])
     if objects[i]['category_name'] not in label_dict:
      label_dict[objects[i]['category_name']] = label_id
      label_id += 1

print(label_dict)

{'Section': 0, 'Other': 1, 'Title': 2, 'Form_key': 3, 'Table_key': 4, 'Form_value': 5, 'Table_value': 6}


## Dataloader

In [ ]:
import math
import torch

# one dimensional feature embedding
def positionalencoding1d(d_model, feature_list):
    """
    :param d_model: dimension of the model
    :param feature_list: length of positions
    :return: length*d_model position matrix
    """
    if d_model % 2 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dim (got dim={:d})".format(d_model))
    pe = torch.zeros(1, d_model)
    feats = torch.tensor(feature_list)
    div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) *
                         -(math.log(10000.0) / d_model)))
    pe[:, 0::2] = torch.sin(feats.float() * div_term)
    pe[:, 1::2] = torch.cos(feats.float() * div_term)
    pe = np.array(pe.tolist())
    return pe

positional_encoding = []

for i in range(100):
    positional_encoding.append(positionalencoding1d(768, i)[0])
positional_encoding = np.array(positional_encoding)

#print(positional_encoding) # (100, 768)

In [ ]:
# unused

def convert_index2onehot(ids, max_len):
  one_hot = [0]*max_len
  for id in ids:
    one_hot[int(id)] = 1
  return one_hot

In [ ]:
# https://github.com/airsplay/lxmert/blob/master/src/tasks/gqa_data.py
# https://huggingface.co/docs/transformers/model_doc/lxmert#lxmert

# code amend if it has coco format
# coco format [x_min, y_min, width, height]
# converted to [x_min, y_min, x_max, y_max]

def normalize_bbox(bbox, page_info):
  x1 = bbox[0]
  y1 = bbox[1]
  x2 = bbox[2]
  y2 = bbox[3]

  width = page_info['width']
  height = page_info['height']

  return [x1/width, y1/height, (x1+x2)/width, (y1+y2)/height]

#### code check

In [ ]:
# val example

doc_info = copy.deepcopy(val_all_feat)
#doc = list(doc_info.keys()) # ['00910692_page-1', '00910694_page-1', ...]

idx=0
key = list(doc_info.keys())[idx] # '00910692_page-1'
doc_info = doc_info[key]
print(doc_info.keys())


texts = []
target_id = []
boxes = []
bert_cls = []
for i in range(len(doc_info["objects"])):
    texts.append(doc_info["objects"][i]["text"])
    #target_id.append(doc_info["objects"][i]["category_name"])
    target_id.append(doc_info["objects"][i]["category_id"])
    boxes.append(copy.deepcopy(doc_info["objects"][i]["bbox"]))
    bert_cls.append(doc_info["objects"][i]["bert_cls"])
#target_id = [label_dict[l] for l in target_id]
text = " ".join(texts)

print(texts)
print(len(texts))
print(target_id)
print(len(target_id))
print(boxes)
print(len(bert_cls))

dict_keys(['name', 'height', 'width', 'objects', 'object_list', 'visual_list'])
['Form 604 \nCorporations Law \nSection 671B', 'Notice of change of interests of substantial holder', 'Previous Notice', 'Present Notice', '1.  Details of substantial holder(1)', 'Previous and present voting power \n2.', '3.  Changes in relevant interests', 'The total number of votes attached to all the voting shares in the company or voting interests in the scheme that the substantial \nholder or an associate (2) had a relevant interest (3) in when last required, and when now required, to give a substantial holding \nnotice in the company or scheme, are as follows:', 'To \nCompany Name/Scheme', 'ACN/ARSN', 'Name', 'ACN/ARSN (if applicable)', 'There was a change in the interests of the  \nsubstantial holder on  ', 'The previous notice was given to the company on ', 'The previous notice was dated', 'Class of securities (4)', 'Person’s votes', 'Person’s votes', 'Voting Power (5)', 'Voting Power', '6.04%', '23

In [ ]:
# val example loop

doc_info = copy.deepcopy(val_all_feat)
#doc = list(doc_info.keys()) # ['00910692_page-1', '00910694_page-1', ...]

idx=0
for idx in range(2):
    key = list(doc_info.keys())[idx] # '00910692_page-1'
    print(key)
    #doc_info = doc_info[key]
    #print(doc_info.keys())
    page_info = doc_info[key]

    texts = []
    target_id = []
    boxes = []
    bert_cls = []
    for i in range(len(page_info["objects"])):
        texts.append(page_info["objects"][i]["text"])
        #target_id.append(page_info["objects"][i]["category_name"])
        target_id.append(page_info["objects"][i]["category_id"])
        boxes.append(copy.deepcopy(page_info["objects"][i]["bbox"]))
        bert_cls.append(page_info["objects"][i]["bert_cls"])
    #target_id = [label_dict[l] for l in target_id]
    text = " ".join(texts)

    print(texts)
    print(len(texts))
    print(target_id)
    print(len(target_id))
    print(boxes)
    print(len(bert_cls))
    #print(bert_cls[0:1])

00910692_page-1
['Form 604 \nCorporations Law \nSection 671B', 'Notice of change of interests of substantial holder', 'Previous Notice', 'Present Notice', '1.  Details of substantial holder(1)', 'Previous and present voting power \n2.', '3.  Changes in relevant interests', 'The total number of votes attached to all the voting shares in the company or voting interests in the scheme that the substantial \nholder or an associate (2) had a relevant interest (3) in when last required, and when now required, to give a substantial holding \nnotice in the company or scheme, are as follows:', 'To \nCompany Name/Scheme', 'ACN/ARSN', 'Name', 'ACN/ARSN (if applicable)', 'There was a change in the interests of the  \nsubstantial holder on  ', 'The previous notice was given to the company on ', 'The previous notice was dated', 'Class of securities (4)', 'Person’s votes', 'Person’s votes', 'Voting Power (5)', 'Voting Power', '6.04%', '23,007,197', '5.03%', '19,169,682', 'ORDINARY \nSHARES', '22/07/20

### custom dataset

In [ ]:
# amended doc_info & page_info

class CustomDataset(Dataset):
    def __init__(self, doc_info, padding_len=100):
        self.tokenizer = tokenizer
        self.doc_info = doc_info
        self.positional_encoding = positional_encoding
        self.padding_len = padding_len

    def __len__(self):
        return len(self.doc_info)

    def __getitem__(self, idx):
        key = list(self.doc_info.keys())[idx]
        page_info = self.doc_info[key]
        print("key", key)

        texts = []
        target_id = []
        boxes = []
        bert_cls = []
        for i in range(len(page_info["objects"])):
            texts.append(page_info["objects"][i]["text"])
            #target_id.append(page_info["objects"][i]["category_id"])
            target_id.append(label_dict[page_info["objects"][i]["category_name"]])
            boxes.append(copy.deepcopy(page_info["objects"][i]["bbox"]))
            bert_cls.append(page_info["objects"][i]["bert_cls"])

        #target_id = [label_dict[l] for l in target_id] # [1, 1, 3, 3, 2, 2, 2, ...]
        text = " ".join(texts)

        #print(texts)
        print("target_id", target_id)
        #print(boxes)
        print("len(bert_cls)", len(bert_cls))

        # Get Fine-Grained Level Information
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=512,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        object_mask = []
        visual_feat = copy.deepcopy(page_info["visual_list"])
        norm_bbox = [normalize_bbox(bbox, page_info) for bbox in boxes]

        print("len(visual_feat)", len(visual_feat))
        #print(visual_feat[0])
        #print(norm_bbox)

        if len(bert_cls) >= self.padding_len:
        #if len(visual_feat) >= self.padding_len:
          bert_cls = bert_cls[:self.padding_len]
          visual_feat = visual_feat[:self.padding_len]
          norm_bbox = norm_bbox[:self.padding_len]
          object_mask = [1]*self.padding_len
          target_id = [-100]*self.padding_len
          #print("if_target_id", target_id)
          #target_id = target_id[:self.padding_len] # not [-100]

        else:
          size = len(visual_feat)
          bert_cls.extend([[0.0]*768]*(self.padding_len-len(bert_cls)))
          visual_feat.extend([[0.0]*2048]*(self.padding_len-len(visual_feat)))
          norm_bbox.extend([[0.0]*4]*(self.padding_len-len(norm_bbox)))
          #object_mask = [1]*len(object_mask)+[0.0]*(self.padding_len-len(object_mask)) #all 0
          object_mask = [1]*size+[0.0]*(self.padding_len-size) # 1 for obj, 0 the rest
          print("object_mask", object_mask)
          target_id.extend([-100]*(self.padding_len-len(target_id)))
          print("else_target_id", target_id)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.float),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'bert_cls': torch.tensor(bert_cls, dtype=torch.float),
            'visual_feat': torch.tensor(visual_feat, dtype=torch.float),
            'target': torch.tensor(target_id, dtype=torch.float),
            'positional_encoding': torch.tensor(self.positional_encoding, dtype = torch.float),
            'object_mask':torch.tensor(object_mask, dtype=torch.float),
            'norm_bbox':torch.tensor(norm_bbox, dtype=torch.float)
        }


## Model Definition

In [ ]:
import torch.nn.functional as F
from transformers import AdamW, AutoModel, set_seed

class New_model(torch.nn.Module):
    def __init__(self):
        super(New_model, self).__init__()
        self.l1 = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased",output_hidden_states = True)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 7) # labels 7


    #def forward(self, ids, mask, token_type_ids, visual_feat, attention_mask, norm_bbox):
    def forward(self, ids, mask, token_type_ids, bert_cls, visual_feat, pos_emb, attention_mask, norm_bbox): #output bert_cls
        visual_attention_mask = attention_mask.to(device, dtype = torch.float)
        output_1 = self.l1(input_ids=ids,
                           attention_mask=mask,
                           token_type_ids=token_type_ids,
                           visual_feats =visual_feat,
                           visual_pos = norm_bbox,
                           visual_attention_mask=visual_attention_mask)

        question = output_1.language_hidden_states[-1] #check how it works
        visual_feat = output_1.vision_hidden_states[-1]
        output = self.pre_classifier(visual_feat)
        output = torch.nn.Tanh()(output)
        output = self.dropout(output)
        output = self.classifier(output)
        return output

In [ ]:
from tqdm.notebook import tqdm

# tokenizer = LxmertTokenizer.from_pretrained('unc-nlp/lxmert-base-uncased')
model = New_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at unc-nlp/lxmert-base-uncased were not used when initializing LxmertModel: ['obj_predict_head.decoder_dict.feat.bias', 'cls.seq_relationship.bias', 'obj_predict_head.decoder_dict.obj.bias', 'cls.predictions.transform.dense.bias', 'obj_predict_head.decoder_dict.attr.bias', 'obj_predict_head.transform.LayerNorm.weight', 'obj_predict_head.transform.LayerNorm.bias', 'answer_head.logit_fc.0.bias', 'answer_head.logit_fc.0.weight', 'cls.seq_relationship.weight', 'obj_predict_head.decoder_dict.feat.weight', 'answer_head.logit_fc.3.bias', 'cls.predictions.transform.LayerNorm.weight', 'obj_predict_head.transform.dense.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.2.bias', 'obj_predict_head.decoder_dict.obj.weight', 'obj_predict_head.decoder_dict.attr.weight', 'obj_predict_head.transform.dense.bias',

New_model(
  (l1): LxmertModel(
    (embeddings): LxmertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LxmertEncoder(
      (visn_fc): LxmertVisualFeatureEncoder(
        (visn_fc): Linear(in_features=2048, out_features=768, bias=True)
        (visn_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (box_fc): Linear(in_features=4, out_features=768, bias=True)
        (box_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer): ModuleList(
        (0-8): 9 x LxmertLayer(
          (attention): LxmertSelfAttentionLayer(
            (self): LxmertAttention(
              (query): Linear(in_fea

## Metrics

In [ ]:
!pip install -q seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
import evaluate
import torch
import numpy as np
from seqeval.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

metric = evaluate.load("seqeval")

In [ ]:
# start from "0"
label_list = list(label_dict.keys())
print("label_dict", label_dict)
print("label_list", label_list)

label_dict {'Section': 0, 'Other': 1, 'Title': 2, 'Form_key': 3, 'Table_key': 4, 'Form_value': 5, 'Table_value': 6}
label_list ['Section', 'Other', 'Title', 'Form_key', 'Table_key', 'Form_value', 'Table_value']


In [ ]:
# original - matching with target_id, but causing an error

# label_list = list(label_dict.keys())
# print("label_dict", label_dict)
# print("label_list", label_list) #label_dict['Title'] = 1

label_dict {'Title': 1, 'Section': 2, 'Other': 3, 'Form_key': 4, 'Form_value': 5, 'Table_key': 6, 'Table_value': 7}
label_list ['Title', 'Section', 'Other', 'Form_key', 'Form_value', 'Table_key', 'Table_value']


In [ ]:
import numpy as np

return_entity_level_metrics = True
# return_entity_level_metrics = False
label_list = list(label_dict.keys())

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[int(p)] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[int(l)] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    report = classification_report(y_true=true_labels, y_pred=true_predictions, digits=4)

    if return_entity_level_metrics:
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
            "classification_report" : report,
        }

## Training

In [ ]:
#train_dataset = CustomDataset(doc_info=train_all_feat)
train_dataset = CustomDataset(doc_info=train_sample)
val_dataset = CustomDataset(doc_info=val_all_feat)
test_dataset = CustomDataset(doc_info=test_all_feat)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [ ]:
#put the model in training mode
loss_function = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)
model.train()

def train(num_train_epochs):
  for epoch in range(num_train_epochs):
      #total_loss = 0
      #print("Epoch:", epoch+1)

      for data in tqdm(train_dataloader):
          # get the inputs;
          ids = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.float)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          labels = data['target'].to(device, dtype = torch.long)
          visual_feats = data['visual_feat'].to(device, dtype = torch.float)
          bert_cls = data['bert_cls'].to(device, dtype = torch.float)
          positional_encoding = data['positional_encoding'].to(device, dtype = torch.float)
          norm_bbox = data['norm_bbox'].to(device, dtype = torch.float)
          object_mask = data['object_mask'].to(device, dtype = torch.float)

          optimizer.zero_grad()
          outputs = model(ids, mask, token_type_ids, bert_cls, visual_feats, positional_encoding, object_mask, norm_bbox)
          #outputs = model (ids, mask, token_type_ids, visual_feats, object_mask, norm_bbox)
          # Change the number of categories
          loss = loss_function(outputs.view(-1, 7), labels.view(-1)) # 7 labels
          loss.backward()
          #total_loss += loss.item()
          optimizer.step()
      #print("Train Loss:", total_loss/len(train_dataloader))


def eval(test_dataloader):
  preds_val = None
  out_label_ids = None
  model.eval()
  #total_loss = 0

  for data in tqdm(test_dataloader):
      with torch.no_grad():
          # get the inputs;
          ids = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.float)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          labels = data['target'].to(device, dtype = torch.long)
          visual_feats = data['visual_feat'].to(device, dtype = torch.float)
          bert_cls = data['bert_cls'].to(device, dtype = torch.float)
          positional_encoding = data['positional_encoding'].to(device, dtype = torch.float)
          norm_bbox = data['norm_bbox'].to(device, dtype = torch.float)
          object_mask = data['object_mask'].to(device, dtype = torch.float)

          optimizer.zero_grad()
          outputs = model(ids, mask, token_type_ids, bert_cls, visual_feats, positional_encoding, object_mask, norm_bbox)
          #outputs = model (ids, mask, token_type_ids, visual_feats, object_mask, norm_bbox)

          #loss = loss_function(outputs.view(-1, 7), labels.view(-1)) # 7 labels
          #total_loss += loss.item()

          if preds_val is None:
            preds_val = outputs.detach().cpu().numpy()
            out_label_ids = data["target"].detach().cpu().numpy()

          else:
            preds_val = np.append(preds_val, outputs.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, data["target"].detach().cpu().numpy(), axis=0)

  #print("Val Loss:", total_loss/len(test_dataloader))
  pprint.pprint(compute_metrics((preds_val, out_label_ids)))
  return compute_metrics((preds_val, out_label_ids))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
current_f1 = 0
best_epoch = 0
optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(2):
  print("Epoch:" epoch+1)
  train(1)
  val_result = eval(val_dataloader)
  if val_result['overall_f1'] > current_f1:
    current_f1 = val_result['overall_f1']
    best_epoch = epoch
    #best_val_result = val_result
    torch.save(model.state_dict(), '/content/formnlu_lxmert_best.pth')

  #print("Best Epoch:", best_epoch+1)
  #pprint.pprint(best_val_result)


Epoch: 1


  0%|          | 0/67 [00:00<?, ?it/s]

key 00828995_page-0
target_id [2, 2, 0, 0, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5]
len(bert_cls) 32
len(visual_feat) 32
object_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
else_target_id [2, 2, 0, 0, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


len(visual_feat) 32
object_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
else_target_id [2, 2, 0, 0, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
key 0162495

...

{'able_key_f1': 0.3322475570032573,
 'able_key_number': 76,
 'able_key_precision': 0.22077922077922077,
 'able_key_recall': 0.6710526315789473,
 'able_value_f1': 0.4096385542168675,
 'able_value_number': 94,
 'able_value_precision': 0.32903225806451614,
 'able_value_recall': 0.5425531914893617,
 'ection_f1': 0.8839506172839506,
 'ection_number': 215,
 'ection_precision': 0.9421052631578948,
 'ection_recall': 0.8325581395348837,
 'itle_f1': 0.9934640522875817,
 'itle_number': 76,
 'itle_precision': 0.987012987012987,
 'itle_recall': 1.0,
 'orm_key_f1': 0.781725888324873,
 'orm_key_number': 82,
 'orm_key_precision': 0.6695652173913044,
 'orm_key_recall': 0.9390243902439024,
 'orm_value_f1': 0.9292035398230089,
 'orm_value_number': 106,
 'orm_value_precision': 0.875,
 'orm_value_recall': 0.9905660377358491,
 'overall_accuracy': 0.8736492103075644,
 'overall_f1': 0.7013662979830839,
 'overall_precision': 0.6069819819819819,
 'overall_recall': 0.8305084745762712}
Epoch: 1


  0%|          | 0/67 [00:00<?, ?it/s]

key 01604855_page-0
target_id [2, 2, 0, 0, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6]
len(bert_cls) 31
len(visual_feat) 31
object_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
else_target_id [2, 2, 0, 0, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

  0%|          | 0/10 [00:00<?, ?it/s]

key 00913924_page-0
target_id [2, 2, 1, 1, 1, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 5, 6, 6, 5, 5]
len(bert_cls) 32
len(visual_feat) 32
object_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
else_target_id [2, 2, 1, 1, 1, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 5, 6, 6, 5, 5, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,

## Evaluation

In [ ]:
state_dict = torch.load('/content/formnlu_lxmert_best.pth')
model.load_state_dict(state_dict)
model.eval()
test_result = eval(test_dataloader)

  0%|          | 0/19 [00:00<?, ?it/s]

key 00918399_page-1
target_id [2, 2, 1, 1, 0, 0, 0, 1, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5]
len(bert_cls) 32
len(visual_feat) 32
object_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
else_target_id [2, 2, 1, 1, 0, 0, 0, 1, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,

In [ ]:
!cp -r /content/formnlu_lxmert_best.pth /content/drive/MyDrive/00_torino_adnlp/